In [1]:
# this is MuZero!
# https://arxiv.org/abs/1911.08265
# It works, but only for CartPole
# Would be happy if it worked for Follower, LunarLander, or Acrobot
# Also, while it's written, it's not using mcts.
# Replace naive_search(which tries n**K) with mcts_search to use

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import tensorflow as tf
import numpy as np
import gym
from tqdm import tqdm, trange
import os,sys
sys.path.append(os.getcwd())

In [3]:
print(tf.__version__)

2.8.0


In [4]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 438963871019809633
 xla_global_id: -1]

In [5]:
print(tf.config.list_physical_devices())
with tf.device('/GPU'):
  a = tf.random.normal(shape=(2,), dtype=tf.float32)
  b = tf.nn.relu(a)

print(a)
print(b)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
tf.Tensor([1.4667784  0.22522028], shape=(2,), dtype=float32)
tf.Tensor([1.4667784  0.22522028], shape=(2,), dtype=float32)


In [6]:
# from tensorflow.python.framework.ops import disable_eager_execution
# disable_eager_execution()

In [7]:
# Make Follower work! Will give interview to anyone who does.
from muzero.follower import Follower
#env = Follower()
env = gym.make("CartPole-v0")
#env = gym.make("MountainCar-v0")
#env = gym.make("LunarLander-v2")
#env = gym.make("Acrobot-v1")

from muzero.model import MuModel
m = MuModel(env.observation_space.shape, env.action_space.n, s_dim=128, K=3, lr=0.001)
print(env.observation_space.shape, env.action_space.n)

from muzero.game import Game, ReplayBuffer
from muzero.mcts import naive_search, mcts_search
replay_buffer = ReplayBuffer(50, 128, m.K)
rews = []

(4,) 2


In [8]:
def play_game(env, m):
  import random
  game = Game(env, discount=0.997)
  while not game.terminal():
    game.env.render()
    cc = random.random()
    if cc < 0.05:
      policy = [1/m.a_dim]*m.a_dim
    else:
      policy = naive_search(m, game.observation, T=1)
    game.act_with_policy(policy)
  return game

In [ ]:
from muzero.model import reformat_batch
import collections

for j in range(30):
  game = play_game(env, m)
  replay_buffer.save_game(game)
  for i in range(20):
    m.train_on_batch(replay_buffer.sample_batch())
  rew = sum(game.rewards)
  rews.append(rew)
  print(len(game.history), rew, collections.Counter(game.history), m.losses[-1][0])

2022-05-17 00:23:01.038410: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


8 8.0 Counter({0: 8}) 5.213472366333008
19 19.0 Counter({0: 11, 1: 8}) 23.832164764404297
119 119.0 Counter({1: 63, 0: 56}) 1879.95654296875
12 12.0 Counter({1: 10, 0: 2}) 1328.507568359375
200 200.0 Counter({0: 101, 1: 99}) 4219.6083984375
33 33.0 Counter({1: 17, 0: 16}) 3474.5498046875
30 30.0 Counter({0: 15, 1: 15}) 3689.006591796875
55 55.0 Counter({0: 30, 1: 25}) 3851.533203125
200 200.0 Counter({0: 103, 1: 97}) 4390.376953125
200 200.0 Counter({0: 101, 1: 99}) 5540.93359375
200 200.0 Counter({1: 101, 0: 99}) 6812.81494140625
200 200.0 Counter({1: 101, 0: 99}) 6100.310546875
200 200.0 Counter({1: 100, 0: 100}) 3947.413818359375


In [ ]:
import matplotlib.pyplot as plt
plt.plot(rews)
plt.yscale('log')
plt.plot([x[0] for x in m.losses])
plt.plot([x[1] for x in m.losses])
plt.plot([x[-3] for x in m.losses])

In [ ]:
# can act?
state = env.reset()
for sn in range(2000):
  p_0 = naive_search(m, state, debug=False, T=0.1)
  #p_0, _ = mcts_search(m, state, 50)
  a_1 = np.random.choice(list(range(len(p_0))), p=p_0)
  _, v_0 = m.ft(m.ht(state))
  
  env.render()
  state,r,done,_ = env.step(a_1)
  print(a_1, v_0, r, p_0)
  if done:
    print("DONE", sn)
    break